In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import pipeline


import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# bert-base-uncased
# distilbert-base-uncased
# roberta-base

def BERT_score(model):
    model_name = model
    if model == 'roberta-base':
        tokenizer = RobertaTokenizer.from_pretrained(model_name, num_labels=1)
        model = RobertaForSequenceClassification.from_pretrained(model_name)
        classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

    else:
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        tokenizer = BertTokenizer.from_pretrained(model_name)
        classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

    return classifier

In [ ]:
# bert-base-uncased

classifier1 = BERT_score('bert-base-uncased')
df1 = pd.read_csv('./crawler/google_news_crawling.csv')
df1['sentiment_label'] = df1['title'].apply(lambda x: classifier1(x)[0]['label'])
df1['sentiment_score'] = df1['title'].apply(lambda x: classifier1(x)[0]['score'])

In [ ]:
# distilbert-base-uncased
classifier2 = BERT_score('distilbert-base-uncased')
df2 = pd.read_csv('../data/google_news_crawling.csv')
df2['sentiment_label'] = df2['title'].apply(lambda x: classifier2(x)[0]['label'])
df2['sentiment_score'] = df2['title'].apply(lambda x: classifier2(x)[0]['score'])

In [ ]:
# roberta-base
# distilbert-base-uncased
classifier3 = BERT_score('roberta-base')
df3 = pd.read_csv('../data/google_news_crawling.csv')
df3['sentiment_label'] = df3['title'].apply(lambda x: classifier3(x)[0]['label'])
df3['sentiment_score'] = df3['title'].apply(lambda x: classifier3(x)[0]['score'])

In [ ]:
df1['sentiment_label'].value_counts()

In [ ]:
df1['sentiment_score'].plot.hist(xlim=(0, 1), bins=10, title='BERT-base-uncased')
plt.savefig('../figure/BERT-base-uncased.png')

In [ ]:
df2['sentiment_label'].value_counts()

In [ ]:
df2['sentiment_score'].plot.hist(xlim=(0, 1), bins=10, title='distilbert-base-uncased')
plt.savefig('../figure/distilbert-base-uncased.png')

In [ ]:
df3['sentiment_label'].value_counts()

In [ ]:
df3['sentiment_score'].plot.hist(xlim=(0, 1), bins=10, title='roberta-base')
plt.savefig('../figure/roberta-base.png')

In [ ]:
df2['sentiment_score'].mean()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df2['sentiment_score_scaled'] = MinMaxScaler().fit_transform(df2['sentiment_score'].values.reshape(-1, 1))

In [ ]:
df2['sentiment_score_scaled'].plot(kind='hist', bins=10, title='distilbert-base-uncased')
plt.savefig('../figure/distilbert-base-uncased_scaled.png')

In [ ]:
noise_temp = df2[['Date', 'sentiment_score_scaled']]

In [ ]:
noise_aggregated = noise_temp.groupby('Date')['sentiment_score_scaled'].apply(list).reset_index()

In [ ]:
max_length = 10
expanded_cols = {
    f'sentiment_{i+1}': pd.Series([x[i] if i < len(x) else 0 for x in noise_aggregated['sentiment_score_scaled']])
    for i in range(max_length)
}


noise_expanded = pd.DataFrame(expanded_cols)
noise_expanded.insert(0, 'Date', noise_aggregated['Date'])

noise_expanded

In [ ]:
noise_expanded['Date'] = pd.to_datetime(noise_expanded['Date'], format='%Y%m%d').astype(str)

In [ ]:
temp_stock = pd.read_csv('../data/AAPL.csv')

In [ ]:
noise_date_info = temp_stock[['Date']].copy()

In [ ]:
noise_data = pd.merge(noise_date_info, noise_expanded, left_on='Date', right_on='Date', how='left')

In [ ]:
noise_data.isna().sum()

In [ ]:
noise_data.fillna(0, inplace=True)

In [ ]:
noise_data.to_csv('../data/noise_data.csv', index=False)

In [ ]:
noise_data